# Рекомендация новых тарифов телеком компании

## 1. Откройте и изучите файл

In [72]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [73]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()
data.head(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1


Датасет содержит информацию о разходовании клиентом пакетов двух тарифных планов. Фрейм занимает 3214 строк информации. Для построения модели классификации, которая будет подбирать подходящий тариф выберем столбец шв_гдекф в качестве целевого признака, а все остальные столбцы с данными будем использованть в качестве признака. Датасет не содержит пропусков, все данные заполнены имеют необходимый тип.

## 2. Разбейте данные на выборки

Разделим исходные данные на на выборки в следующем процентном соотношении:
* data_train - обучающая выборка (60%)
* data_test - тестовая выборка (20%)
* data_valid - валидационная аыборка (20%)

Использовали модель для деления 3:1:1 так как, помимо валидационной выборки выделяли так же тестовую выборку для дальнейшего тестирования поведения модели.

In [74]:
data_train, data_test_valid = train_test_split(data, test_size=0.4, random_state=12345)
data_test, data_valid = train_test_split(data_test_valid, test_size=0.5, random_state=12345)

## 3. Исследуйте модели

Сохраненим признаки в отдельных переменных. features - все признаки, кроме целевого признака (название тарифа), target — целевой признак, столбец is_ultra (где 1 - Ультра-тариф, 0 - Смарт).

In [75]:
features_train = data_train.drop(columns=['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(columns=['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(columns=['is_ultra'], axis=1)
target_test = data_test['is_ultra']

Построим модель дерева решений. И найдем качество этой модели (accuracy).

In [76]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)

valid_predictions = model.predict(features_valid)
valid_accuracy = accuracy_score(target_valid, valid_predictions)

print("Валидационная выборка:", valid_accuracy)

Валидационная выборка: 0.7309486780715396


Проверка качества модели на валидационной модели дало результат примерно 0,73, что, в принципе соответствует желаемому. Попробуем подобрать такие гиперпараметры модели, чтобы значение accuracy было выше 0,75. Для этого проанализируем как меняется accuracy при изменении максимальной глубины дерева.

In [77]:
for depth in range(1, 9):
    model = DecisionTreeClassifier(max_depth = depth, random_state=12345)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    valid_accuracy = accuracy_score(target_valid, valid_predictions)
    print("max_depth =", depth, "accuracy =", valid_accuracy)

max_depth = 1 accuracy = 0.7356143079315708
max_depth = 2 accuracy = 0.7744945567651633
max_depth = 3 accuracy = 0.7791601866251944
max_depth = 4 accuracy = 0.7744945567651633
max_depth = 5 accuracy = 0.7838258164852255
max_depth = 6 accuracy = 0.776049766718507
max_depth = 7 accuracy = 0.7993779160186625
max_depth = 8 accuracy = 0.7931570762052877


Из результатов видим, что наилучшее accuracy тогда, когда глубина модели равнна 7, используем это значение, когда будем проверять работу модели на тестовой выборке.

Построим модель Случайного леса. Так же посмотрим значения качества accuracy.

In [78]:
model = RandomForestClassifier(random_state=12345, n_estimators=3)
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
train_accuracy = accuracy_score(target_train, train_predictions)

valid_predictions = model.predict(features_valid)
valid_accuracy = accuracy_score(target_valid, valid_predictions)

print("Обучающая выборка:", train_accuracy)
print("Валидационная выборка:", valid_accuracy)

Обучающая выборка: 0.9507261410788381
Валидационная выборка: 0.7573872472783826


Проверка качества модели на валидационной модели дало результат примерно 0,75, что, в принципе соответствует желаемому. Гиперпараметром в данном случае является n_estimators (количество деревьев в лесу), обучали, указав его равным трем. Попробуем посмотреть другие значения данного гиперпараметра, чтобы увеличить значение accuracy.

In [79]:
for estimator in range(10, 100, 20):
    for depth in range(1, 16, 2):
        for state in range(12345, 37036, 12345):
            model = RandomForestClassifier(n_estimators=estimator, max_depth=depth, random_state=state)
            model.fit(features_train, target_train)
            valid_predictions = model.predict(features_valid)
            valid_accuracy = accuracy_score(target_valid, valid_predictions)
            print("n_estimators =", estimator, "max_depth =", depth, "random_state =", state, "accuracy =", valid_accuracy)

n_estimators = 10 max_depth = 1 random_state = 12345 accuracy = 0.7402799377916018
n_estimators = 10 max_depth = 1 random_state = 24690 accuracy = 0.749611197511664
n_estimators = 10 max_depth = 1 random_state = 37035 accuracy = 0.7356143079315708
n_estimators = 10 max_depth = 3 random_state = 12345 accuracy = 0.776049766718507
n_estimators = 10 max_depth = 3 random_state = 24690 accuracy = 0.7791601866251944
n_estimators = 10 max_depth = 3 random_state = 37035 accuracy = 0.7776049766718507
n_estimators = 10 max_depth = 5 random_state = 12345 accuracy = 0.7900466562986003
n_estimators = 10 max_depth = 5 random_state = 24690 accuracy = 0.7978227060653188
n_estimators = 10 max_depth = 5 random_state = 37035 accuracy = 0.7900466562986003
n_estimators = 10 max_depth = 7 random_state = 12345 accuracy = 0.8009331259720062
n_estimators = 10 max_depth = 7 random_state = 24690 accuracy = 0.8040435458786936
n_estimators = 10 max_depth = 7 random_state = 37035 accuracy = 0.7993779160186625
n_esti

Из результатов подбора видим, что наилучшее accuracy тогда, когда:

n_estimators = 10 max_depth = 9 random_state = 12345 accuracy = 0.8133748055987559

Используем эти значения гиперпараметров, когда будем проверять работу модели на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

Проверим модель Дерево решений на тестовой выборке, найдем значение качества accuracy.

In [80]:
model = DecisionTreeClassifier(random_state=12345, max_depth = 7)
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
train_accuracy = accuracy_score(target_train, train_predictions)

test_predictions = model.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)

print("Обучающая выборка:", train_accuracy)
print("Тестовая выборка:", test_accuracy)

Обучающая выборка: 0.8558091286307054
Тестовая выборка: 0.7822706065318819


Модель хорошо показала себя при проверке на тестовой выборке. Значение accuracy равно примерно 0,78, что удовлетворяет условиям задачи.

Проверим модель Случайный лес на тестовой выборке, так же найдем значение качества модели (accuracy).

In [81]:
model = RandomForestClassifier(n_estimators = 10, max_depth = 9, random_state = 12345)
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
train_accuracy = accuracy_score(target_train, train_predictions)

test_predictions = model.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)

print("Обучающая выборка:", train_accuracy)
print("Тестовая выборка:", valid_accuracy)

Обучающая выборка: 0.8729253112033195
Тестовая выборка: 0.7962674961119751


Модель хорошо показала себя при обучении на данных тестовой выборке. Значение accuracy равно примерно 0,78, что удовлетворяет условиям задачи.

После изменения гиперпараметров, на которых обучаем модель, ее качество улучшилось до, примерно, 0,8.

## 5. (бонус) Проверьте модели на адекватность

Проверим модель на адекватность методом DummyClassifier(), данный метод подставляет для обучения рандомные значения и проверяет какое значение accuracy покажет модель. Если качество модели на рандомных значениях выше accuracy нашей модели (Случайного леса или Дерева решений), то наша модель не прошла проверку на адекватность.

In [82]:
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(features_train, target_train)
predictions_clf = dummy_clf.predict(features_train)
dummy_clf.score(target_train, predictions_clf)

0.5191908713692946

Accuracy модели DummyClassifier() примерно на уровне 0,49. Это дает нам основание полагать, что наша модель адекватна и готова принимать данные для анализа подходящего для клиента тарифа.